In [43]:
import numpy as np
import sys
print(sys.version)
print(np.__version__)

3.6.6 | packaged by conda-forge | (default, Jul 26 2018, 11:48:23) [MSC v.1900 64 bit (AMD64)]
1.15.1


Ein nützliches Tool zur konstruktion der Hessenbergtransformation sind sogenannte [Householdtransformationen(https://de.wikipedia.org/wiki/Householdertransformation).

In [22]:
def householdvector(a):
    v = a + np.sign(a[0])*np.linalg.norm(a)*np.eye(len(a))[0,:]
    return v/np.linalg.norm(v)

def householdtransform(a):
    v = householdvector(a)
    return np.eye(len(a)) - 2*(np.outer(v,v))

Beispiel zu illustration, dass die Transformation in die Richtung des ersten Einheitsvektors spiegelt.

In [55]:
a = np.array([1,2,3,4,5,6])
np.round(householdtransform(a) @ a,5)

array([-9.53939, -0.     ,  0.     , -0.     ,  0.     ,  0.     ])

Eine Beschreibung des Hessenbergalgorithmus kann als teil der [QR Algorithmus Wikipediaseite](https://de.wikipedia.org/wiki/QR-Algorithmus#Transformation_auf_Hessenberg-Form) gefunden werden

Zunächst wird eine Hilfsfunktion konstruiert die die $P_k$ transformationen berechnet.

In [27]:
def partialHessenbergTransformation(a,N):
    householdDimension = len(a)
    identiyDimension = N - len(a)
    ht = householdtransform(a)
    return np.block([[np.eye(identiyDimension),np.zeros((identiyDimension,householdDimension))],
                    [np.zeros((householdDimension,identiyDimension)),ht]])
    

Der Hessenbergalgorithmus berechnet die $P_k$ iterativ und gibt ihr Produkt als gesammttransformation zurueck|

In [37]:
def hessenbergTransformation(matrix):
    d = matrix.shape[0]
    currentTransformation = np.eye(d)
    currentMatrix = matrix
    for i in range(d-2):
        a = currentMatrix[i+1:d,i]
        partialTransformation = partialHessenbergTransformation(a,d)
        currentMatrix = partialTransformation @ currentMatrix @ np.linalg.inv(partialTransformation)
        currentTransformation = partialTransformation @ currentTransformation
    return currentTransformation

Beispiel

In [56]:
m = np.array([[1,2,3,1],[4,5,6,3],[7,2,8,9],[8,2,20,9]])
m

array([[ 1,  2,  3,  1],
       [ 4,  5,  6,  3],
       [ 7,  2,  8,  9],
       [ 8,  2, 20,  9]])

In [52]:
t = hessenbergTransformation(m)
t

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.35218036, -0.61631563, -0.70436073],
       [ 0.        , -0.20119433, -0.68512901,  0.70008505],
       [ 0.        , -0.91405133,  0.38826959,  0.11728977]])

In [53]:
np.round(t @ m @ np.linalg.inv(t),5)

array([[  1.     ,  -3.25767,  -1.75769,  -0.546  ],
       [-11.35782,  23.68992,   6.30211,  -5.69861],
       [ -0.     ,  -3.83335,  -5.14264,   3.67677],
       [  0.     ,   0.     ,   2.00392,   3.45272]])